# Most probable outcomes

In [12]:
import pandas as pd

# Load the dataset
df = pd.read_excel('shorterData.xlsx')

# List of iDoc columns
idoc_columns = ['iDoc02', 'iDoc03', 'iDoc04', 'iDoc05', 'iDoc06', 'iDoc07', 'iDoc08', 'iDoc09', 'iDoc10']

def sort_idocs(row):
    # Extract non-empty iDoc values into a list
    idoc_values = [row[col] for col in idoc_columns if pd.notna(row[col]) and row[col] != '']
    
    # Sort the list alphabetically
    sorted_idocs = sorted(idoc_values)
    
    # Fill remaining columns with empty strings if there are fewer than 9 iDoc values
    sorted_idocs += [''] * (len(idoc_columns) - len(sorted_idocs))
    
    # Create a dictionary with sorted iDoc values assigned back to the iDoc columns
    return pd.Series({col: sorted_idocs.pop(0) for col in idoc_columns})

# Apply the sorting to each row and replace the iDoc columns with the sorted values
df[idoc_columns] = df.apply(sort_idocs, axis=1)

# Save the new sorted dataset to a new Excel file
df.to_excel('alphaData.xlsx', index=False)

print("iDoc values have been sorted alphabetically and saved to 'sortedData.xlsx'.")


iDoc values have been sorted alphabetically and saved to 'sortedData.xlsx'.


In [5]:
import pandas as pd

df = pd.read_excel('shorterData.xlsx', dtype={'Commodity Code': str})

# Define the iDoc columns
idoc_columns = ['iDoc02', 'iDoc03', 'iDoc04', 'iDoc05', 'iDoc06', 'iDoc07', 'iDoc08', 'iDoc09', 'iDoc10']

# Remove all whitespace from each entry in the iDoc columns
for col in idoc_columns:
    df[col] = df[col].str.replace(r'\s+', '', regex=True)

# Save the cleaned DataFrame back to a new Excel file
df.to_excel('cleanedData.xlsx', index=False)
print(df.head())

                    ImpName            ExName Commodity Code  \
0  Lensbower, Gregory L Esq   US Clipper Inc.     7308100000   
1            Mcmahan, Ben L   US Clipper Inc.     4820200000   
2    Rowley/hansell Petetin   US Clipper Inc.     4820200000   
3    Rowley/hansell Petetin   US Clipper Inc.     4820200000   
4               Appbyte Ltd  Demo Company Ltd     0105993000   

  CountryofOrigin_key                          iDoc02  \
0                  US                             NaN   
1                  US                             NaN   
2                  US                             NaN   
3                  US                             NaN   
4                  FR  C505~Guaranteenotrequired~CC~~   

                     iDoc03          iDoc04 iDoc05 iDoc06 iDoc07 iDoc08  \
0                       NaN             NaN    NaN    NaN    NaN    NaN   
1                       NaN             NaN    NaN    NaN    NaN    NaN   
2                       NaN             NaN    

In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import joblib

# Load and preprocess the dataset
df = pd.read_excel('alphaData.xlsx', dtype={'Commodity Code': str})
df.fillna('Missing', inplace=True)  # Replace NaNs with 'Missing'
df = df.astype(str)

X = df[['ImpName', 'ExName', 'Commodity Code', 'CountryofOrigin_key']]
y = df[['iDoc02', 'iDoc03', 'iDoc04', 'iDoc05', 'iDoc06', 'iDoc07', 'iDoc08', 'iDoc09', 'iDoc10']]

# One-Hot Encoding with Unknown variable handling
encoder_X = OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first')
X_encoded = encoder_X.fit_transform(X)
joblib.dump(encoder_X, 'onehot_encoder_X_with_unknown.pkl')

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Build the neural network model
def build_complex_model(output_size):
    model = Sequential([
        Dense(256, input_dim=X_train.shape[1], activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(output_size, activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Dictionary to store models and label encoders
models = {}
label_encoders = {}

# Train a model for each iDoc column
for col in y_train.columns:
    le = LabelEncoder()
    
    # Replace missing values with 'Missing' before encoding
    y_train[col] = y_train[col].replace('', 'Missing')
    y_test[col] = y_test[col].replace('', 'Missing')
    
    # Fit the label encoder on the training data
    y_train_encoded = le.fit_transform(y_train[col].astype(str))
    
    # Transform the test data, filtering out any rows where the label is unseen
    valid_indices = y_test[col].isin(le.classes_)
    y_test_filtered = y_test[col][valid_indices]
    y_test_encoded = le.transform(y_test_filtered)
    X_test_filtered = X_test[valid_indices]
    
    # Compute class weights to handle the "Missing" data weighting
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_train_encoded),
        y=y_train_encoded
    )
    class_weights_dict = dict(enumerate(class_weights))
    
    # Build the model
    model = build_complex_model(output_size=len(le.classes_))
    
    # Train the model with early stopping and class weights
    model.fit(X_train, y_train_encoded, 
              epochs=20, 
              batch_size=64, 
              validation_data=(X_test_filtered, y_test_encoded), 
              callbacks=[early_stopping],
              class_weight=class_weights_dict)  # Include class weights here
    
    # Save the model and label encoders
    model.save(f'nn_model_{col}.h5')
    joblib.dump(le, f'label_encoder_{col}.pkl')

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test_filtered, y_test_encoded)
    print(f"Accuracy for {col}: {accuracy:.2f}")
    print("-" * 30)


Epoch 1/20
105/105 [==============================] - 1s 3ms/step - loss: 5.5562 - accuracy: 0.0149 - val_loss: 5.3622 - val_accuracy: 0.0000e+00
Epoch 2/20
105/105 [==============================] - 0s 2ms/step - loss: 5.0338 - accuracy: 0.0921 - val_loss: 5.3262 - val_accuracy: 0.0120
Epoch 3/20
105/105 [==============================] - 0s 2ms/step - loss: 4.5296 - accuracy: 0.1493 - val_loss: 4.9746 - val_accuracy: 0.0764
Epoch 4/20
105/105 [==============================] - 0s 2ms/step - loss: 4.1441 - accuracy: 0.1707 - val_loss: 4.3728 - val_accuracy: 0.1533
Epoch 5/20
105/105 [==============================] - 0s 2ms/step - loss: 3.7615 - accuracy: 0.1798 - val_loss: 3.7347 - val_accuracy: 0.3001
Epoch 6/20
105/105 [==============================] - 0s 2ms/step - loss: 3.4423 - accuracy: 0.2103 - val_loss: 3.2311 - val_accuracy: 0.3764
Epoch 7/20
105/105 [==============================] - 0s 2ms/step - loss: 3.1917 - accuracy: 0.2547 - val_loss: 2.9259 - val_accuracy: 0.3710
Ep

/Users/ayush/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/20
105/105 [==============================] - 1s 2ms/step - loss: 5.2042 - accuracy: 0.0223 - val_loss: 5.0098 - val_accuracy: 0.0090
Epoch 2/20
105/105 [==============================] - 0s 2ms/step - loss: 4.3783 - accuracy: 0.0668 - val_loss: 4.9236 - val_accuracy: 0.0084
Epoch 3/20
105/105 [==============================] - 0s 2ms/step - loss: 3.8140 - accuracy: 0.1175 - val_loss: 4.7029 - val_accuracy: 0.0192
Epoch 4/20
105/105 [==============================] - 0s 2ms/step - loss: 3.3277 - accuracy: 0.1507 - val_loss: 4.2652 - val_accuracy: 0.1455
Epoch 5/20
105/105 [==============================] - 0s 2ms/step - loss: 2.9548 - accuracy: 0.1722 - val_loss: 3.7644 - val_accuracy: 0.2634
Epoch 6/20
105/105 [==============================] - 0s 2ms/step - loss: 2.5770 - accuracy: 0.1886 - val_loss: 3.3344 - val_accuracy: 0.2886
Epoch 7/20
105/105 [==============================] - 0s 2ms/step - loss: 2.3001 - accuracy: 0.2113 - val_loss: 3.1033 - val_accuracy: 0.3115
Epoch 

/Users/ayush/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/20
105/105 [==============================] - 1s 2ms/step - loss: 5.2746 - accuracy: 0.0170 - val_loss: 4.9996 - val_accuracy: 0.0024
Epoch 2/20
105/105 [==============================] - 0s 2ms/step - loss: 4.4690 - accuracy: 0.0504 - val_loss: 5.0159 - val_accuracy: 0.0168
Epoch 3/20
105/105 [==============================] - 0s 2ms/step - loss: 3.8519 - accuracy: 0.0829 - val_loss: 4.7940 - val_accuracy: 0.0481
Epoch 4/20
105/105 [==============================] - 0s 2ms/step - loss: 3.4355 - accuracy: 0.0973 - val_loss: 4.4089 - val_accuracy: 0.1318
Epoch 5/20
105/105 [==============================] - 0s 2ms/step - loss: 3.0724 - accuracy: 0.1081 - val_loss: 4.0063 - val_accuracy: 0.1504
Epoch 6/20
105/105 [==============================] - 0s 2ms/step - loss: 2.6147 - accuracy: 0.1157 - val_loss: 3.7332 - val_accuracy: 0.1655
Epoch 7/20
105/105 [==============================] - 0s 2ms/step - loss: 2.3733 - accuracy: 0.1288 - val_loss: 3.5538 - val_accuracy: 0.2178
Epoch 

...

/Users/ayush/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Accuracy for iDoc08: 0.00
------------------------------
Epoch 1/20
105/105 [==============================] - 1s 2ms/step - loss: 3.6195 - accuracy: 0.0513 - val_loss: 3.0371 - val_accuracy: 0.0000e+00
Epoch 2/20
105/105 [==============================] - 0s 2ms/step - loss: 2.5677 - accuracy: 0.0496 - val_loss: 3.2048 - val_accuracy: 5.9916e-04
Epoch 3/20
105/105 [==============================] - 0s 2ms/step - loss: 2.0721 - accuracy: 0.0702 - val_loss: 3.1973 - val_accuracy: 5.9916e-04
Epoch 4/20
105/105 [==============================] - 0s 2ms/step - loss: 1.8827 - accuracy: 0.0776 - val_loss: 3.1460 - val_accuracy: 0.0042
Epoch 5/20
105/105 [==============================] - 0s 2ms/step - loss: 1.6732 - accuracy: 0.0825 - val_loss: 3.2653 - val_accuracy: 0.0491
Epoch 6/20
105/105 [==============================] - 0s 2ms/step - loss: 1.8593 - accuracy: 0.0834 - val_loss: 3.4107 - val_accuracy: 0.0719
Epoch 7/20
105/105 [==============================] - 0s 2ms/step - loss: 1.479

/Users/ayush/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Accuracy for iDoc09: 0.00
------------------------------
Epoch 1/20
105/105 [==============================] - 1s 2ms/step - loss: 2.5203 - accuracy: 0.1046 - val_loss: 2.2170 - val_accuracy: 0.0018
Epoch 2/20
105/105 [==============================] - 0s 2ms/step - loss: 1.5588 - accuracy: 0.0937 - val_loss: 2.3384 - val_accuracy: 0.0024
Epoch 3/20
105/105 [==============================] - 0s 2ms/step - loss: 0.8302 - accuracy: 0.0894 - val_loss: 2.3519 - val_accuracy: 0.0114
Epoch 4/20
105/105 [==============================] - 0s 2ms/step - loss: 0.7031 - accuracy: 0.0946 - val_loss: 2.2626 - val_accuracy: 0.0436
Epoch 5/20
105/105 [==============================] - 0s 2ms/step - loss: 0.5364 - accuracy: 0.1072 - val_loss: 2.1764 - val_accuracy: 0.1381
Epoch 6/20
105/105 [==============================] - 0s 2ms/step - loss: 0.4841 - accuracy: 0.1155 - val_loss: 2.1420 - val_accuracy: 0.2104
Epoch 7/20
105/105 [==============================] - 0s 2ms/step - loss: 0.5407 - accuracy

/Users/ayush/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import joblib

# Prediction with new unseen data
def predict_new_data(new_data):
    encoder_X = joblib.load('onehot_encoder_X_with_unknown.pkl')
    idoc_columns = ['iDoc02', 'iDoc03', 'iDoc04', 'iDoc05', 'iDoc06', 'iDoc07', 'iDoc08', 'iDoc09', 'iDoc10']
    
    # One-Hot Encode the new data using the encoder
    new_data_encoded = encoder_X.transform(new_data)
    
    predictions = {}
    
    # Iterate over each iDoc column to make predictions
    for col in idoc_columns:
        model = tf.keras.models.load_model(f'nn_model_{col}.h5')
        le = joblib.load(f'label_encoder_{col}.pkl')
        
        # Make predictions
        encoded_pred = model.predict(new_data_encoded)
        prob_pred = encoded_pred[0]
        
        # Store the predictions sorted by probability
        sorted_preds = sorted(zip(le.classes_, prob_pred), key=lambda item: item[1], reverse=True)
        predictions[col] = sorted_preds
    
    return predictions


new_data = pd.DataFrame({
    'ImpName': ['Appbyte Ltd'], 
    'ExName': ['Demo Company Ltd'], 
    'Commodity Code': ['8529101100'], 
    'CountryofOrigin_key': ['PL']
})

predictions = predict_new_data(new_data)
if predictions:
    for col, preds in predictions.items():
        print(f"Predictions for {col}:")
        for label, prob in preds:
            if prob > 0.005:  
                print(f"  {label}: {prob:.4f}")
        print("-" * 30)

1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 33ms/step
Predictions for iDoc02:
  C119~4320~AE~TEST~: 0.3932
  C505~GBCGU02511510000720200206120129~CC~~: 0.3113
  C505~Guaranteenotrequired~CC~ ~: 0.1978
  C505~GBCGU02511510000720200206120129~CC~ ~: 0.0184
  C064~3211~AC~Test~: 0.0134
------------------------------
Predictions for iDoc03:
  U110~Invoice~AE~Invoice~: 0.6505
  C506~GBDPO3712600~~~: 0.2213
  C505~Guaranteenotrequired~CC~ ~: 0.0565
  C505~GBCGU55320273485220191113093459~CC~ ~: 0.0111
  C672~Invoice~AE~test~: 0.0095
------------------------------
Predictions for iDoc04:
  U110~Invoice~AE~Invoice~: 0.3889
  Missing: 0.1674
  N935~INV0003~AC~ ~: 0.0239
  C506~GBDPO1108654~ ~ ~: 0.0228
  C672~Invoice~AE~test~: 0.0177
  Y053~3221~ ~test~: 0.0154
  C506~GBDPO1108654~~~: 0.0146
  N853~7655~AE~test~: 0.0129
  Y922~3111~ ~test~: 0.0120
  C601~GBIPO02511510000720200206120129~~~: 0.0114
  C505~Guaranteenotrequired~CC~ ~: 0.0099
  C601~ GBIPO89645889501520190613093448~~~: 0.0075
  Y929~454

In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import joblib

# Prediction with new unseen data
def predict_new_data(new_data):
    # Load the encoder and models
    encoder_X = joblib.load('onehot_encoder_X_with_unknown.pkl')
    idoc_columns = ['iDoc02', 'iDoc03', 'iDoc04', 'iDoc05', 'iDoc06', 'iDoc07', 'iDoc08', 'iDoc09', 'iDoc10']
    
    # One-Hot Encode the new data using the encoder
    new_data_encoded = encoder_X.transform(new_data)
    
    predictions = {}
    
    # Iterate over each iDoc column to make predictions
    for col in idoc_columns:
        model = tf.keras.models.load_model(f'nn_model_{col}.h5')
        le = joblib.load(f'label_encoder_{col}.pkl')
        
        # Make predictions
        encoded_pred = model.predict(new_data_encoded)
        prob_pred = encoded_pred[0]  # Assuming single data point for prediction
        
        # Store the predictions sorted by probability
        sorted_preds = sorted(zip(le.classes_, prob_pred), key=lambda item: item[1], reverse=True)
        predictions[col] = sorted_preds[:5]  # Only keep the top 5 predictions
    
    return predictions


new_data = pd.DataFrame({
    'ImpName': ['Appbyte Ltd'], 
    'ExName': ['Demo Company Ltd'], 
    'Commodity Code': ['3917330010'], 
    'CountryofOrigin_key': ['PL']
})

predictions = predict_new_data(new_data)
if predictions:
    for col, preds in predictions.items():
        print(f"Top 5 Predictions for {col}:")
        for label, prob in preds:
            if prob > 0.005:  
                print(f"  {label}: {prob:.4f}")
        print("-" * 30)


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 33ms/step
Top 5 Predictions for iDoc02:
  C505~Guaranteenotrequired~CC~~: 0.5098
  C506~GBDPO3712600~~~: 0.0476
  1207~20231020~AG~~: 0.0266
  1207~20230529~AG~~: 0.0227
  1207~20240215~AG~~: 0.0196
------------------------------
Top 5 Predictions for iDoc03:
  U110~Invoice~AE~Invoice~: 0.8049
  9120~RefNumber~AE~Reason~: 0.0163
  Y032~4444~~test~: 0.0146
  C644~2133~AE~test1~: 0.0093
  C506~GBDPO1108654~~~: 0.0083
------------------------------
Top 5 Predictions for iDoc04:
  C119~4320~AE~TEST~: 0.7521
  U110~Invoice~AE~Invoice~: 0.0746
  Missing: 0.0513
  C064~3211~AC~Test~: 0.0463
  C672~Invoice~AE~test~: 0.0089
------------------------------
Top 5 Predictions for iDoc05:
  Missing: 0.3074
  C505~Guaranteeenotrequired~CC~~: 0.0632
  Y926~3221~~test~: 0.0488
  U110~Invoice~AE~Invoice~: 0.0403
  C505~Guaranteenotrequired~CC~~: 0.0346
------------------------------
Top 5 Predictions for iDoc06:
  Missing: 0.0974
  Y922~3111~~test~: 0.0503
  Y93

# Accuracy with top 5 

In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import joblib

# Prediction with new unseen data
def predict_new_data(new_data, actual_values):
    encoder_X = joblib.load('onehot_encoder_X_with_unknown.pkl')
    idoc_columns = ['iDoc02', 'iDoc03', 'iDoc04', 'iDoc05', 'iDoc06', 'iDoc07', 'iDoc08', 'iDoc09', 'iDoc10']
    
    # One-Hot Encode the new data using the encoder
    new_data_encoded = encoder_X.transform(new_data)
    
    predictions = {}
    
    # Iterate over each iDoc column to make predictions
    for col in idoc_columns:
        model = tf.keras.models.load_model(f'nn_model_{col}.h5')
        le = joblib.load(f'label_encoder_{col}.pkl')
        
        # Make predictions
        encoded_pred = model.predict(new_data_encoded)
        prob_pred = encoded_pred[0] 
        
        # Store the predictions sorted by probability
        sorted_preds = sorted(zip(le.classes_, prob_pred), key=lambda item: item[1], reverse=True)
        predictions[col] = sorted_preds[:5]  # Only keep the top 5 predictions
    
    # Evaluate top-5 accuracy
    correct_predictions = 0
    total_predictions = len(idoc_columns)
    
    for col in idoc_columns:
        top_preds = [label for label, prob in predictions[col]]
        actual_value = actual_values[col].iloc[0]  # Get the actual value for the corresponding iDoc column
        
        if actual_value in top_preds:
            correct_predictions += 1
            print(f"Actual {col}: {actual_value} was in the top 5 predictions")
        else:
            print(f"Actual {col}: {actual_value} was NOT in the top 5 predictions")
    
    top_5_accuracy = correct_predictions / total_predictions
    print(f"Top-5 accuracy: {top_5_accuracy:.2f}")
    
    return predictions

# Example usage:
new_data = pd.DataFrame({
    'ImpName': ['Appbyte Ltd'], 
    'ExName': ['Demo Company Ltd'], 
    'Commodity Code': ['204505900'], 
    'CountryofOrigin_key': ['US']
})

actual_values = pd.DataFrame({
    'iDoc02': ['9120~2211~AE~test3~'],  
    'iDoc03': ['C505~GBCGU02511510000720200206120129~CC~ ~'],
    'iDoc04': ['C506~GBDPO3712600~~ ~'],
    'iDoc05': ['C644~1210~AE~ ~'],
    'iDoc06': ['N853~3431~AE~ ~'],
    'iDoc07': ['Missing'],
    'iDoc08': ['Missing'],
    'iDoc09': ['Missing'],
    'iDoc10': ['Missing']
})

predictions = predict_new_data(new_data, actual_values)
if predictions:
    for col, preds in predictions.items():
        print(f"Top 5 Predictions for {col}:")
        for label, prob in preds:
            if prob > 0.005:  
                print(f"  {label}: {prob:.4f}")
        print("-" * 30)


1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 35ms/step
Actual iDoc02: 9120~2211~AE~test3~ was in the top 5 predictions
Actual iDoc03: C505~GBCGU02511510000720200206120129~CC~ ~ was in the top 5 predictions
Actual iDoc04: C506~GBDPO3712600~~ ~ was NOT in the top 5 predictions
Actual iDoc05: C644~1210~AE~ ~ was in the top 5 predictions
Actual iDoc06: N853~3431~AE~ ~ was in the top 5 predictions
Actual iDoc07: Missing was NOT in the top 5 predictions
Actual iDoc08: Missing was NOT in the top 5 predictions
Actual iDoc09: Missing was NOT in the top 5 predictions
Actual iDoc10: Missing was in the top 5 predictions
Top-5 accuracy: 0.56
Top 5 Predictions for iDoc02:
  9120~2211~AE~test3~: 0.4262
  1207~20230614 ~AG~~: 0.2208
  1207~20230613 ~AG~~: 0.1626
  C505~GBCGU02511510000720200206120129~CC~~: 0.0287
  C505~GBCGU0835173300020200828141500~CC~ ~: 0.0149
------------------------------
Top 5 Predictions for iDoc03:
  C505~Guaranteenotrequired~CC~ ~: 0.3996
  C505~GBCGU02511510000720200206120129~

In [2]:
model = tf.keras.models.load_model('nn_model_iDoc02.h5')

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model
with open('nn_model_iDoc02.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /var/folders/j8/s3w557v576v0dtnghwcjfxm40000gn/T/tmpcwgkxruh/assets


INFO:tensorflow:Assets written to: /var/folders/j8/s3w557v576v0dtnghwcjfxm40000gn/T/tmpcwgkxruh/assets
2024-09-13 12:12:24.328375: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-09-13 12:12:24.328399: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-09-13 12:12:24.328848: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/j8/s3w557v576v0dtnghwcjfxm40000gn/T/tmpcwgkxruh
2024-09-13 12:12:24.329973: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-09-13 12:12:24.329978: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/j8/s3w557v576v0dtnghwcjfxm40000gn/T/tmpcwgkxruh
2024-09-13 12:12:24.332517: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2024-09-13 12:12:24.333520: I tensorflow/cc/saved_model/load